# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [3]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...
...,...,...
64,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AI...
65,기타,AICE 시험관련 문의는 어디에 해야하나요?\nhelp@aice.study 으로 문...
66,기타,국민취업지원제도를 신청하고 싶습니다.\n국민취업지원제도 신청과 관련 사항은 거주지 ...
67,기타,국민취업지원제도를 안해도 교육 참여되나요?\n국민취업제도를 신청하지 않아도 KT 에...


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",
                    embedding_function = embeddings
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [ ]:
ids = database.get()
database.delete(ids=ids['ids'])

In [5]:
documents = [Document(page_content=row['QA'], metadata={'category':row['구분']}) for idx, row in data.iterrows()]
database.add_documents(documents)

['b0048476-912b-44cb-8702-a0a3a8ed5504',
 '8f3ddfac-4fdd-4920-b236-41b0220c8c2b',
 '93d918bf-34bb-41e0-b5ea-a022c2cd41f1',
 '94da3fdd-2b0f-442a-9124-e2224791824d',
 '15d40645-1780-41c8-9db0-1231938eb19e',
 'bc6f856b-7078-4b07-ad51-20b3633abe8c',
 'a898994f-c531-4686-8218-ec2027962c17',
 '1382773c-31cd-417d-9c80-76c59474d396',
 'b23431e6-a3f8-4f35-acca-57fa7fabb4b6',
 '3e646deb-a107-4d33-9834-d6f0f1eaf73a',
 '08b5bb89-aa56-4e6d-9297-a2e0abda69c1',
 '8a6595c7-f6de-4224-9042-21c5c11e5db9',
 'e7e5911e-c84f-4004-8be5-d3806f29f9f0',
 '338fb8ff-d546-4f79-9672-53f0a888d3fa',
 '66ea278a-6190-4da7-b73e-0208f08fe79a',
 '534a52b5-1b88-45cd-82d3-6f054a8beb26',
 '7a2b705d-a887-4286-8230-94b2529014ac',
 'cca508f2-d736-499e-a356-6ecd32be73a0',
 'cd507892-71be-4e7e-949c-ccd905a3a658',
 '6fac6603-6e13-4721-88ba-0f4577af4ea6',
 '9270a981-d63a-441f-b556-600271d3b270',
 '1d8a7124-2eeb-4962-b971-90a5235a1958',
 '8f593fb4-281a-44a2-b803-1c94b13a9f05',
 'fb721bc9-5516-406a-9f44-6870640ee0ba',
 '5bd34107-d75f-

* 입력된 데이터 조회

In [6]:
database.get()

{'ids': ['0098f160-da1b-4092-b0eb-0b8a35a7ade1',
  '08b5bb89-aa56-4e6d-9297-a2e0abda69c1',
  '0dbd634d-5379-460b-a759-e2bf101539f9',
  '1382773c-31cd-417d-9c80-76c59474d396',
  '15d40645-1780-41c8-9db0-1231938eb19e',
  '1991f8a1-5d30-471f-b088-8ce0e76e0c54',
  '1aa0d025-073c-4030-8aa8-717817ec2178',
  '1d8a7124-2eeb-4962-b971-90a5235a1958',
  '20f20d58-7faa-4ee5-9ab2-90982aa58924',
  '22503023-b3e4-4d14-8d54-5302b23bcd03',
  '30ac5c35-faf6-40c8-bc57-588148dd5fe8',
  '338fb8ff-d546-4f79-9672-53f0a888d3fa',
  '3a203184-4186-4c05-b69f-f04b516c2b70',
  '3e646deb-a107-4d33-9834-d6f0f1eaf73a',
  '3fa67144-0eee-4a31-a95c-2b3c51482a92',
  '480252a2-21f7-4ad7-be9b-080784270293',
  '4b022fc8-4877-4e56-ac69-05c9abc07c98',
  '534a52b5-1b88-45cd-82d3-6f054a8beb26',
  '5410a8dd-80fe-41ed-9e5b-865ed9152620',
  '553a7f06-36bc-4515-b07f-fd47751c57c3',
  '5740005a-b883-4a0a-93b6-d654692c7c6d',
  '585ccf8b-2925-49a6-bfef-8ca730360fe1',
  '5bd34107-d75f-458f-9a9c-fcf8c5020bdc',
  '60e21220-9787-4401-9be7-

크롤링1: 네이버 블로그 검색

In [7]:
import os
import sys
import urllib.request
import json
import re

# 네이버 API 애플리케이션에서 확인 (웹크롤링 강의 때 생성함)
client_id = ""
client_secret = ""

encText = urllib.parse.quote("에이블스쿨")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=50&sort=sim"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

blog_url = set()

if(rescode==200):
    response_body = response.read()
    response_dict = json.loads(response_body.decode('utf-8'))
    items = response_dict['items']
    for item_i in range(0, len(items)):
        link = items[item_i]['link']
        blog_url.add(link)
else:
    print("Error Code:" + rescode)

In [8]:
blog_url

{'https://blog.naver.com/1etskoo/223433046723',
 'https://blog.naver.com/26wkdtnrud26/223415829186',
 'https://blog.naver.com/44seo/223438612738',
 'https://blog.naver.com/asj9661/223330649674',
 'https://blog.naver.com/baenan1/223361235419',
 'https://blog.naver.com/baenan1/223367080906',
 'https://blog.naver.com/baenan1/223437179256',
 'https://blog.naver.com/baenan1/223464757087',
 'https://blog.naver.com/cctv5060/223395638907',
 'https://blog.naver.com/datalytics/223430848071',
 'https://blog.naver.com/ditkddus/223415810783',
 'https://blog.naver.com/ditkddus/223423854993',
 'https://blog.naver.com/ditkddus/223430510952',
 'https://blog.naver.com/dnehtks123/223279443445',
 'https://blog.naver.com/dnehtks123/223302824340',
 'https://blog.naver.com/dnehtks123/223329179777',
 'https://blog.naver.com/dnehtks123/223330905135',
 'https://blog.naver.com/fiddms0924/223373058596',
 'https://blog.naver.com/fiddms0924/223460479327',
 'https://blog.naver.com/fiddms0924/223460503723',
 'https:/

In [9]:
from bs4 import BeautifulSoup
import requests

text_list = []

for url_b in blog_url:
    x = url_b.split('/')
    url=f'https://blog.naver.com/PostView.naver?blogId={x[-2]}&logNo={x[-1]}&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false'
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select('.se-module-text')
    text = [element.text.strip() for element in elements]
    text_list.append('\n'.join(text))

In [10]:
text_list

['KT 에이블스쿨 3차 미니프로젝트\n오랜만에 쓰는 포스팅이다.\n합격할 줄 몰랐는데... 나중에 6기모집시기에 지원 관련 포스팅도 써야겠다.\n\u200b나는 현재 다니던 직장을 퇴사하고 KT 에이블스쿨이라는 교육을 듣고 있다.그냥 듣는것보다, 어디엔가 기록하며 들으면 더 수업에 열심히 참여할 수 있을 것 같았다.(백수면 간절해야지...)\u200b1차 때부터 글을 써보려고 했으나...생각보다 너무 바빴어서(핑계) ... 지금부터라도 포스팅을 올려본다.\u200b그럼 시작!!!!!!! 가보자고 !!!\u200b\u200b\u200b사실 난 대학교 때 프로젝트를 해본적이 없다. 공대 특성상 1,2학년은 특별한 조별 과제가 없었고, 3,4학년은 코로나 비대면이었다.심지어 전 직장에서도 협업보다는 개인적인 업무가 많은 편인 나 였다!\n\u200b협업이 익숙하지 않았떤 만큼,1차 미니프로젝트부터 부족한 점이 많았고, 나는 그 점을 메꿔가기 위해 많이 노력하기로 했고, 노력했다.왜냐하면 협업역량을 키우는 것은 이번 교육 나의 가장 중요한 목표 중 하나이기 때문이다. \u200b내가 1,2차 프로젝트를 통해 느낀 부족한 점은의견 표출에 소극적이었던 점코드가 생각나지 않아, 그 때 그때 이전 문서를 뒤져봐야 했던 점\u200b곰곰히 생각해본 결과, 1번의 원인은 자신감이 부족했기 때문이라고 생각했다. 그 자신감이 부족한 원인은, 내 실력이 부족한것(2번)이 가장 컸던 것 같다. 코드를 따라치기도 벅찼기에,  팀원들의 생각과 대화 흐름을 이해하고 쫓아가기가 너무 힘들었다. \u200b그래서 집에와서 연습했다.1. 에이블러님들들이 공유해주신 프로젝트 코드를 하나씩 따라가면서 다른 사람들의 생각의 흐름을 배우고자 하였고,2. 자주 쓰는 코드를 모아놓은 문서를 따로 만들었다.\n에이블스쿨에는 뛰어난 에이블러분들이 정말 많다.프로젝트 후에 코드파일을 공유해주셔서 항상 감사하다.언젠간 나도 프로젝트에서 좋은 성과를 얻어 다른 사람에게 공유할 만한 코드를 만들고 싶다.\u200b

In [11]:
documents = [Document(page_content=text, metadata={'category': '블로그'}) for text in text_list]
database.add_documents(documents)

['feb398bc-0758-491e-97eb-dd461aa2b944',
 '1e5ab8f3-001f-489e-8602-7ceeae7d94a3',
 '6ae0a809-90f3-4ac1-ab87-41aa42ce3ed7',
 '6a5b590e-4fd9-4d9e-b2f0-0d23b2542c05',
 '331b21b8-f1e3-4a25-870d-ae0c9b1bf512',
 'bd5285a5-2a15-46b4-97af-d2bec9d95a67',
 '221a258c-0b02-454b-9511-e1a874d579a6',
 '9cbe0c9b-0e8a-47f2-9d07-6b8f9e0928cc',
 '452437df-5b25-487a-9c21-54a1b17939de',
 'c792cb91-d886-473b-98de-6c391f7eb870',
 '6a434182-81b3-423e-a0df-f1a9ffd6e708',
 '129e1b6b-52f1-43dd-bec4-326110abf0bb',
 '914aefb5-57b4-4138-80c2-f0a5f7b54897',
 'e9a5c4b1-04d6-48f2-9506-b5ccf260f074',
 '2e22af6b-4a9e-4237-890a-7efaff3a8116',
 '7c18779b-265f-4fd5-a942-0587e5bf9b9a',
 'cc9a7b4a-7233-4f03-b6aa-c5bcbfd64ed3',
 'c2dfccde-1737-4e47-9e8c-06e3cddea49a',
 '97ec965b-ebe3-4af0-9b23-dcad7f97ec3d',
 'c328bf4a-bbc4-47ef-8bc4-77725045cbf6',
 'cfe68425-0165-4c6c-b81f-6fceb758710d',
 '5a7a2f0a-7220-4c50-88e5-f4bf8a465f4c',
 '27d76744-b6e6-404c-bb1d-c5bf8fee7172',
 '19b554f4-e2ee-4fa2-b43b-ee4a78a58785',
 '8d87fac4-99e1-

크롤링2: 네이버 뉴스 (애매하게 내용 바꾼 중복이 너무 많음..ㅠㅠ 사용 어려울 듯)

In [12]:
client_id = ""
client_secret = ""

encText = urllib.parse.quote("에이블스쿨")
url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&display=50&sort=sim"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

text_news = set()

if(rescode==200):
    response_body = response.read()
    response_dict = json.loads(response_body.decode('utf-8'))
    items = response_dict['items']
    for item_i in range(0, len(items)):
        remove_tag = re.compile('<.*?>')
        description = re.sub(remove_tag, '', items[item_i]['description'])
        text_news.add(description)
else:
    print("Error Code:" + rescode)

In [13]:
text_news

{'20일 KT 분당사옥에서 진행된 KT 에이블스쿨 5기 입교식 모습.  KT(대표 김영섭)가 분당사옥에서 에이블스쿨 5기 교육생 600여 명이 참여한 가운데 20일 온라인으로 입교식을 열고 본격적인 교육을 시작했다고 21일 밝혔다.... ',
 '20일 오후 KT 분당사옥에서 진행된 KT 에이블스쿨 5기 입교식 모습 (사진=KT) KT가 분당사옥에서 KT 에이블스쿨 5기 교육생 600여 명이 참여한 가운데 20일 온라인으로 입교식을 열고 본격적인 교육을 시작했다고 21일 밝혔다.... ',
 '20일 오후 KT 분당사옥에서 진행된 KT 에이블스쿨 5기 입교식 모습 (사진=KT) KT는 분당사옥에서 KT 에이블스쿨 5기 교육생 600여 명이 참여한 가운데 20일 오후 온라인으로 입교식을 열고 본격적인 교육을 시작했다고 21일... ',
 'KT 분당사옥에서 진행된 KT 에이블스쿨 5기 입교식 모습 KT는 20일 분당사옥에서 KT 에이블스쿨 5기 교육생 600여 명이 참여한 가운데 온라인으로 입교식을 열고 본격적인 교육을 시작했다고 21일 밝혔다. 에이블스쿨은 KT가... ',
 'KT 에이블스쿨 5기 입교식 KT가 에이블스쿨 5기 입교식을 진행했다. KT 에이블스쿨은 디지털 인재 육성의 일환으로, 정부와 함께 진행하는 고용 연계 교육 프로그램이다. KT는 34세 이하 미취업자 중 4년제 대학 졸업자... ',
 'KT 에이블스쿨 5기 입교식 현장 (사진=KT) KT(대표 김영섭)는 KT 에이블스쿨 5기 교육생 600여명과 온라인으로 입교식을 열고 교육을 시작했다고 21일 밝혔다. 에이블스쿨은 KT가 정부와 진행하는 디지털 경쟁력 제고... ',
 "KT(030200)가 분당 사옥에서 KT '에이블스쿨 5기' 교육생 600여 명이 참여한 가운데 온라인으로 입교식을 열고 본격적인 교육을 시작했다고 21일 밝혔다. 에이블스쿨은 KT가 정부와 함께 기업 실무형 디지털 인재를... ",
 'KT(대표 김영섭)는 20일 오후 분당사옥에서 KT 에이블스쿨 5기 교육생 60

In [14]:
documents = [Document(page_content=text, metadata={'category': '뉴스'}) for text in text_news]
database.add_documents(documents)

['0c295b0d-b739-49b3-8313-d1060bf94764',
 'aa3228bb-d095-450c-9233-32e872993dc0',
 '54335c6d-3cfe-4371-87f8-c296822cb8f5',
 '52348ce3-d452-4a2c-8143-23ef1dd07159',
 '9a82a42d-2888-4e30-9015-c4b2dd6c2ccd',
 'a95092c4-7b62-43d7-a824-6ff44f8c5f62',
 '96005029-cda3-461a-86b9-e20dfe156594',
 '2717b2ef-a760-49d5-8fc6-678f53733031',
 '31ef828e-477f-4593-bdfb-0da76788d716',
 '47240648-879a-4d3f-9408-b8ff1550d6ad',
 '4d6af1be-8c19-4414-86d7-a9d8d830bdbc',
 '5d1eade6-1bf0-4711-9575-b97964d86ea3',
 '1e89edf0-5e60-4355-884d-85ccedd79dce',
 '10194fb2-e655-4cfc-89ba-ded1958d0e11',
 'ffe0fd1e-ba50-4cd7-81b3-ecbf0a5adc0f',
 '2730716a-552d-43be-8394-1d33ad109e19',
 'f47ae2cd-65d3-4716-a217-6d37044732ac',
 'b25c27dc-f067-4b4c-b13f-3affd57c10a2',
 'b292bb2f-8659-49cf-bd1e-4da1b46a81ae',
 'e97979a4-95f3-4be8-9ba5-6f6d4cdb9fbf',
 '89036d5d-d583-4bbd-943f-1df6bed7121b',
 '08485baa-05fd-49d5-a633-1bf244678ea1',
 '97cec855-c9af-43e0-9ade-bd1c0c33f3a8',
 '575bcca7-f2cd-4755-a572-46cef757bb1c',
 'c5cb2d63-5cec-

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [15]:
path = path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [16]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

* 모델 사용 및 이력 확인

In [17]:
while True:
    query = input('질문 > ')
    query = query.strip()
    
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    print(f'질문 : {query}')
    print('-' * 20)
    
    if len(query) == 0:
        break
    
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)
    
    doc = database.similarity_search_with_score(query, k = k)
    
    conn = sqlite3.connect(path)
    df = pd.DataFrame({'datetime': [dt], 'query': [query], 
                       'sim1' : [round(doc[0][1], 5)], 'sim2' : [round(doc[1][1], 5)], 'sim3' : [round(doc[2][1], 5)], 
                       'answer' : [result["answer"]]})
    df.to_sql('history', conn, if_exists='append', index=False)
    conn.close()

질문 : 에이블스쿨이 뭐야
--------------------
답변 : 에이블스쿨은 KT의 'AI 인재 양성 경험과 노하우'를 확장하여, 대한민국 청년들을 '기업 실무형 AI/DX인재'로 양성하고, 일자리 창출에 기여하는 교육 프로그램입니다. AI 트랙과 DX 트랙으로 나뉘며, AI 개발자와 DX 컨설턴트로서의 역량을 갖출 수 있도록 교육이 이루어집니다. 또한, 취업 지원 및 자소서 컨설팅 등의 서비스도 제공됩니다.
질문 : AI 트랙에선 뭘 배워
--------------------
답변 : AI 트랙에서는 머신러닝, 딥러닝, Object Detection(YOLO 모델), Pandas AI 등과 같은 내용을 배우게 됩니다. 머신러닝과 딥러닝을 포함하여 다양한 기술과 알고리즘에 대한 학습이 이루어지며, 심화된 주제들을 다루게 됩니다.
질문 : 지원 조건이 뭐야
--------------------
답변 : 지원 조건은 미취업자에게 제공되며, 훈련비용은 고용노동부와 KT가 전액 부담합니다. 현재 재직자는 지원이 불가능하며, 취업 여부가 확인될 예정입니다.
질문 : 교육 전에 퇴사할건데 괜찮아?
--------------------
답변 : 퇴사 후에도 훈련 수강에는 문제가 없습니다. 다만, 실업급여와 훈련장려금은 중복 수급이 불가능하므로, 실업급여 지급이 종료된 후에 훈련장려금이 지급됩니다.
질문 : 훈련장려금이 뭐야
--------------------
답변 : 훈련장려금은 긴 기간의 국비과정을 수강하는 분들에게 경제적 부담을 덜어주기 위해 교통비와 식비 명목으로 제공되는 지원금을 말합니다.
질문 : 얼마나 줘
--------------------
답변 : 2023년의 경우, 훈련장려금은 월 최대 116,000원이 지급됩니다. 특별훈련수당이 추가되면 최대 316,000원이 될 수 있습니다.
질문 : 
--------------------


In [18]:
conn = sqlite3.connect(path)
df = pd.read_sql('SELECT * FROM history', conn)
display(df)
conn.close()

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 12:58:49,에이블스쿨이 뭐야,0.28567,0.31248,0.32426,"에이블스쿨은 KT의 'AI 인재 양성 경험과 노하우'를 확장하여, 대한민국 청년들을..."
1,2,2024-06-04 12:59:05,AI 트랙에선 뭘 배워,0.34742,0.34955,0.35970,"AI 트랙에서는 머신러닝, 딥러닝, Object Detection(YOLO 모델),..."
2,3,2024-06-04 12:59:40,지원 조건이 뭐야,0.31735,0.33353,0.34370,"지원 조건은 미취업자에게 제공되며, 훈련비용은 고용노동부와 KT가 전액 부담합니다...."
3,4,2024-06-04 13:00:02,교육 전에 퇴사할건데 괜찮아?,0.29986,0.30304,0.30931,"퇴사 후에도 훈련 수강에는 문제가 없습니다. 다만, 실업급여와 훈련장려금은 중복 수..."
4,5,2024-06-04 13:00:31,훈련장려금이 뭐야,0.22400,0.26745,0.26986,훈련장려금은 긴 기간의 국비과정을 수강하는 분들에게 경제적 부담을 덜어주기 위해 교...
5,6,2024-06-04 13:00:39,얼마나 줘,0.38368,0.40752,0.43153,"2023년의 경우, 훈련장려금은 월 최대 116,000원이 지급됩니다. 특별훈련수당..."
